# **Normalizzazione dei Nomi Aziendali**

## **Introduzione**

Nel processo di deduplicazione e record linkage, è essenziale normalizzare i nomi aziendali per evitare variazioni dovute a abbreviazioni, formati o errori di battitura. In questa fase, implementiamo una strategia per rendere i nomi uniformi e comparabili.

---

## **Step 1: Importazione delle Librerie e del DataFrame**

In [4]:
import re
import pandas as pd
import unidecode

AZIENDE_CSV = '../aziende.csv'
IMPIEGATI_CSV = '../impiegati.csv'

In [5]:
companies_df = pd.read_csv(AZIENDE_CSV)

/tmp/ipykernel_23801/2611636423.py:1: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  companies_df = pd.read_csv(AZIENDE_CSV)


---

## **Step 2: Definizione della Mappa di Sostituzione**
Utilizziamo un dizionario per standardizzare suffissi comuni nei nomi aziendali.
### **Questo va sicuramente migliorato magari passando tutta la prima colonna a chat**

In [69]:
REPLACEMENTS = {
    r"&": "",
    r"(?<=\s|,|\.)inc\.?\b": "",
    r"(?<=\s|,|\.)ltd\b": "",
    r"(?<=\s|,|\.)llc\b": "",
    r"(?<=\s|,|\.)plc\b": "",
    r"(?<=\s|,|\.)co\.?\b": "",  # Handles "co." and "co,"
    r"(?<=\s|,|\.)corp\.?\b": "",
    r"(?<=\s|,|\.)srl\b": "",
    r"(?<=\s|,|\.)spa\b": "",
    r"(?<=\s|,|\.)ag\b": "",
    r"(?<=\s|,|\.)sa\b": "",
    r"(?<=\s|,|\.)ab\b": "",
    r"(?<=\s|,|\.)nv\b": "",
    r"(?<=\s|,|\.)ou\b": "",
    r"(?<=\s|,|\.)as\b": "",
    r"(?<=\s|,|\.)gmbh\b": "",
    r"(?<=\s|,|\.)holding\b": "",
    r"(?<=\s|,|\.)holdings\b": "",
    r"(?<=\s|,|\.)corporation\b": "",
    r"(?<=\s|,|\.)industries\b": "",
    r"(?<=\s|,|\.)group\b": "",
    r"(?<=\s|,|\.)company limited\b": "",
    r"(?<=\s|,|\.)technology\b": "",
    r"(?<=\s|,|\.)solutions\b": "",
    r"(?<=\s|,|\.)systems\b": "",
    r"(?<=\s|,|\.)services\b": "",
    r"(?<=\s|,|\.)public company\b": "",

    # New patterns to remove dotted abbreviations
    r"\bs\.?p\.?a\.?\b": "",  # Handles "S.p.A." and variations
    r"\bs\.?r\.?l\.?\b": "",  # Handles "S.R.L."
    r"\bs\.?a\.?\b": "",      # Handles "S.A."
    r"\bn\.?v\.?\b": "",      # Handles "N.V."
}


---

## **Step 3: Funzione di Normalizzazione**

Questa funzione applica una serie di trasformazioni per ottenere nomi uniformi:

- Conversione in minuscolo
- Rimozione di accenti
- Eliminazione di caratteri speciali
- Sostituzione dei suffissi tramite `REPLACEMENTS`

In [66]:
def normalize_name(name):
    """Normalizza i nomi delle aziende."""
    if pd.isna(name) or not isinstance(name, str):
        return ""
    
    # 1. Rimuove accenti e caratteri speciali
    name = unidecode.unidecode(name)
    
    # 2. Minuscolo e rimozione spazi extra
    name = name.lower().strip()
    
    # 3. Rimuove testo tra virgolette
    name = re.sub(r'"([^"]+)"','', name) # 3. Rimuove testo tra virgolette
    
    # 4. Applica le sostituzioni dalla mappa REPLACEMENTS
    for pattern, replacement in REPLACEMENTS.items():
        name = re.sub(pattern, replacement, name)
    
    # 5. Pulisce caratteri inutili
    name = re.sub(r"\s*,+\s*", " ", name)
    name = re.sub(r"\s*\.+\s*", " ", name) 
    
    # 6. Sostituisce spazi multipli con uno solo
    name = re.sub(r'\s+', ' ', name)  
    
    # 7. Eliminare punteggiatura indesiderata alla fine 
    name = re.sub(r"[.,\-]+$", "", name)
    
    return name.strip()

---

## **Step 4: Applicazione alla Colonna 'company_name'**

Eseguiamo la normalizzazione sulla colonna contenente i nomi aziendali.

In [70]:
companies_df['normalized_name'] = companies_df['company_name'].apply(normalize_name)
companies_df['modified'] = companies_df['normalized_name'] != companies_df['company_name']

---

## **Step 5: Visualizzazione e salvataggio del Risultato**

Mostriamo i primi risultati per verificare la trasformazione
e salva tutto in un csv

In [71]:
num_modified = companies_df['modified'].sum()
total_entries = len(companies_df)
percentage_modified = (num_modified / total_entries) * 100

print(f"Entries modificate: {num_modified} su {total_entries} ({percentage_modified:.2f}%)")

modified_entries = companies_df[companies_df['modified']][['company_name', 'normalized_name']]
display(modified_entries.sample(25))

companies_df[['company_name', 'normalized_name', 'modified']].to_csv('normalized_company_name.csv', index=False)

Entries modificate: 39932 su 76808 (51.99%)


,company_name,normalized_name
5642,asian marine services public company ltd,asian marine
27091,good beer club ltd,good beer club
32766,infineum italia srl,infineum italia
76039,NaN,
61924,spirit technology solutions ltd.,spirit
47168,oncolytics biotech inc.,oncolytics biotech
27889,grenadier holdings plc,grenadier
21647,equitable group,equitable
33153,"inspire medical systems, inc.",inspire medical
11848,calisen plc,calisen


---

## **Conclusioni**

Questo approccio garantisce una normalizzazione efficace dei nomi aziendali, riducendo le variazioni dovute a differenze di scrittura, abbreviazioni e formati. È facilmente estendibile aggiornando la mappa `REPLACEMENTS` con nuove regole di sostituzione.

